In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print("current working directiry is [" + os.getcwd() + "]")
_colab_dir = "/content/drive/MyDrive/Colab Notebooks/ResTran/examples/notebooks"
os.chdir(_colab_dir)
print("current working directiry is [" + os.getcwd() + "]")

In [ ]:
# Imports
import torch
cuda = torch.cuda.is_available()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append("../../semi-supervised")
sys.path.append("../../")


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class NN(nn.Module):
    def __init__(self, dims, dropout_rate):

        super(NN, self).__init__()

        [x_dim, y_dim, h_dim] = dims
        self.dropout_rate = dropout_rate
        neurons = [x_dim, *h_dim, y_dim]
        linear_layers = [nn.Linear(neurons[i-1], neurons[i]) for i in range(1, len(neurons))]

        self.hidden = nn.ModuleList(linear_layers)

    def forward(self, x):
        for layer in self.hidden:
            x = F.relu(layer(x))
            x = F.dropout(x, p=self.dropout_rate, training=self.training)
        return x

In [ ]:
#### data

import importlib
import datautils
import datasets
importlib.reload(datautils)
importlib.reload(datasets)
from datautils import get_data_krylov

# Only use 10 labelled examples per class
# The rest of the data is unlabelled.
labelled, unlabelled, validation, test = get_data_krylov(batch_size=128,
                                                                    labels_per_class=977,
                                                                    validation_ratio=0.25,
                                                                    dataset_name='pubmed',
                                                                    norm_flag=False,
                                                                    gamma=1,
                                                                    b = 0,
                                                                    #dataset_path='/content/drive/My Drive/Colab Notebooks/ssldeep/semi-supervised-pytorch-master/examples/notebooks/data')
                                                                    dataset_path='data/',
                                                                    n_workers = 0)

alpha = 0.1 * (len(unlabelled) + len(labelled)) / len(labelled)
print(alpha)
def binary_cross_entropy(r, x):
    return -torch.sum(x * torch.log(r + 1e-8) + (1 - x) * torch.log(1 - r + 1e-8), dim=-1)


In [ ]:
###model definition
import models
importlib.reload(models)
from models import VATLoss


y_dim = 3
#h_dim = [2048, 1024]
h_dim = [1024, 512]
#h_dim = [512, 256]
n_feature = 500

dropout_rate = 0.03
lr = 1e-4
momentum = 0.4
xi = 0.053
eps = 0.049
ip = 1

alpha = 1
#alpha = 1 * (len(unlabelled) + len(labelled)) / len(labelled)

model = NN([n_feature, y_dim, h_dim],dropout_rate=dropout_rate)
print(model)

#optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))
if cuda: model = model.cuda()

NN(
  (hidden): ModuleList(
    (0): Linear(in_features=500, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=3, bias=True)
  )
)
